### Data Preprocessing

1. Initialization
2. Combination with region dataset
3. Feature generation
4. Removal of misleading data
5. Column transformation
6. Dropping of unnecessary columns

Output:
- One dataset ready to be fed into numerical-based models.
- An additional dataset for text based models e.g. XGBoost.

**The data used for this project is confidential and owned by TFA so all outputs have been cleared out.**

**Initialization**

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Load original dataset
df = pd.read_csv('WPAC_2023_CandidateRecords.csv')

In [ ]:
# Change ComfirmOffer_date to ConfirmOffer_date
df.rename(columns={'ComfirmOffer_date': 'ConfirmOffer_date', 'Progress_4_Comfirm_Offer': 'Progress_4_Confirm_Offer'}, inplace=True)

**Combine with regional data**

In [ ]:
# load regional dataset
regional_df = pd.read_csv('Region_Cost.csv')

# reindex regional dataset
regional_df = regional_df.reset_index()
regional_df = pd.melt(regional_df, id_vars='Region', value_vars=['2016', '2017', '2018', '2019', '2020', '2021', '2022','2023'])
regional_df['variable']= regional_df['variable'].astype(int)

# merge datasets by column 'Region' and 'Match_region'
df = pd.merge(df, regional_df, left_on=['Match_region', 'App_year'], right_on=['Region', 'variable'], how='left')
df = df.rename(columns={'value': 'Cost'})

**Feature generation**

In [ ]:
# Application completion time
df['App_submit_date'] = df['App_submit_date'].replace(np.NAN,0)
df['App_submit_date'] = df['App_submit_date'].str.replace('-', '').astype(np.float).astype("Int32")
df['App_start_date'] = df['App_start_date'].replace(np.NAN,0)
df['App_start_date'] = df['App_start_date'].str.replace('-', '').astype(np.float).astype("Int32")
df['app_completion_days'] = df["App_submit_date"] - df["App_start_date"]

In [ ]:
# Amount of fields filled
self_filled_cols = ["career_level", "UG_school_name", "UG_school_selectivity", "UG_major_byGroup", "UG_major_bySchool",
                    "UG_major_minor_STEM","UG_GPA", "UG_sports","UG_PellGrant","LIC_served_self_report",
                    "Leadership_role","family_responsibility","Preference_Region_level","Preference_Region_rank",
                    "Preferences_Subject_level"]
amount_of_cols = len(self_filled_cols)
df["filled_cols_count"] = amount_of_cols-df[self_filled_cols].isna().sum(axis = 1)

In [ ]:
# Binary amount of fields filled (second option)
# If field_cols_count is 15, then 1, if 14 or less, then 0
df['filled_cols_count_bin'] = np.where(df['filled_cols_count'] == 15, 1, 0)

In [ ]:

# Steps completed
binary_cols = ["Progress_1_Invite_Intrvw", "Progress_2_Complete_Intrvw", "Progress_3_Accepted_toCorp", "Progress_4_Confirm_Offer", "Progress_5_Start_1stDay", "Progress_6_Complete_2yrs"]
for col in binary_cols:
    df[col] = df[col].apply(lambda x: 1 if x == True or x == "TRUE" else 0)

df["steps_completed"] = df["Progress_1_Invite_Intrvw"] + df["Progress_2_Complete_Intrvw"] + df["Progress_3_Accepted_toCorp"] +df["Progress_4_Confirm_Offer"] + df["Progress_5_Start_1stDay"]


In [ ]:

# Financial gap
prev_LIC_level = {np.NAN: 0, "LEVEL1": 1, "LEVEL2": 2, "LEVEL3": 3}
df["prev_LIC_level"] = df["LIC_served_rating"].map(prev_LIC_level)
min = df.Cost.min()
max = df.Cost.max()
diff = (max-min)/3
df["financial_gap"] = df.apply(lambda x: x["Cost"] - (min+(3-x["prev_LIC_level"])*(diff)), axis=1)


In [ ]:
# Average dimensions score
df["avg_dimension_score"] = df[["SelectionDimension_1", "SelectionDimension_2", "SelectionDimension_3", "SelectionDimension_4"]].mean(axis = 1, skipna=True)

In [ ]:
# Offer delay days
df['ConfirmOffer_date'] = df['ConfirmOffer_date'].str.replace('-', '').astype(np.float).astype("Int32")
df["ConfirmOffer_date"]=df["ConfirmOffer_date"].replace(np.NAN, 0)
df["offer_delay_days"] = df["ConfirmOffer_date"] - df["App_submit_date"]
df["offer_delay_days"] = df["offer_delay_days"].clip(lower=0)

In [ ]:
# Create application number column
df['app_number'] = df.groupby('user_ID')['user_ID'].transform('count')

In [ ]:
# Ivy league school
ivy_list = ['Brown University', 'Columbia University', 'Cornell University', 'Dartmouth College', 'Harvard University', 'University of Pennsylvania', 'Princeton University', 'Yale University']
# Create a new column called 'Ivy_league' and set it to 1 if the school in UG_school_name is in the ivy_list
df['Ivy_league'] = df['UG_school_name'].apply(lambda x: 1 if x in ivy_list else 0)

**Removing problematic candidate data**

- Candidates with null values in 2YearCompletion
- Candidates who started at 2019-2020 (COVID)
- Candidates who started at 2022 (haven't finished yet)

In [ ]:
df = df.dropna(subset=['Progress_6_Complete_2yrs'])
df.drop(df[df.App_year == 2019].index, inplace=True)
df.drop(df[df.App_year == 2020].index, inplace=True)
df.drop(df[df.App_year == 2022].index, inplace=True)

----------------------------------------------------------------------------------------- Export text-category dataset -----------------------------------------------------------------------------------------

In [ ]:
"""
text_df = df[["career_level", "UG_school_name", "UG_school_selectivity","UG_major_byGroup","UG_major_bySchool","UG_PellGrant","LIC_served_self_report","LIC_served_rating"]]

text_df.to_csv('text_df.csv', index=False)
"""

**Column transformation**

In [ ]:
# Encode career level
career_level = {'Junior': 0, 'Co-terming Senior': 1, 'Undergrad': 2, 'Grad Student': 3, 'Professional': 4}
df['career_level'] = df['career_level'].map(career_level)

# Encode school selectivity
df["UG_school_selectivity"] = df["UG_school_selectivity"].fillna("Unknown")
UG_school_selectivity = {"Unknown": 0, "Least Selective": 1, "Less Selective": 2, "Selective": 3, "More Selective": 4, "Most Selective": 5}
df["UG_school_selectivity"] = df["UG_school_selectivity"].map(UG_school_selectivity)

# Encode UG STEM, UG Sports, Leadership_role
binary_cols = ["UG_major_minor_STEM", "UG_sports", "Leadership_role", "family_responsibility"]
for col in binary_cols:
    df[col] = df[col].apply(lambda x: 1 if x == "Y" else 0)

# Encode Pellgrant
UG_PellGrant = {"NO": 0, "Y": 0, "PARTIAL": 1, "MAXIMUM": 2}
df["UG_PellGrant"] = df["UG_PellGrant"].map(UG_PellGrant)


In [ ]:
# High profile candidate
mean = df['UG_GPA'].mean()
df['high_profile'] = df.apply(lambda x: ( x['Ivy_league'] + (x['UG_school_selectivity']== 5) + (x['UG_GPA'] > mean)), axis=1)

**Drop unnecessary columns**

In [ ]:
df = df.drop(["user_ID", "App_year", "UG_school_name", "UG_major_byGroup", "UG_major_bySchool",
              "LIC_served_self_report", "LIC_served_rating",
              "Preference_Region_level", "Preference_Region_rank", "Preferences_Subject_level",
              "SelectionDimension_1", "SelectionDimension_2", "SelectionDimension_3", "SelectionDimension_4",
              "Match_region", "Teach_region", "Teach_subject"], axis=1)

--------------------------------------------------------------------------------------------- Export main dataset ---------------------------------------------------------------------------------------------

In [ ]:
df.to_csv('processed_df.csv', index=False)